Data cleanning: remove duplicate images and resize image to 256,256 and RGB mode

In [27]:
from data_preprocessing.data_cleanning import Data_cleanning
data_cleanning = Data_cleanning()
data_cleanning.remove_duplicated_image('ai_generated_images/')
data_cleanning.remove_duplicated_image('human_generated_images/')


checking image number 0 
checking image number 1 
checking image number 2 
checking image number 3 
checking image number 4 
checking image number 5 
checking image number 6 
checking image number 7 
checking image number 8 
checking image number 9 
checking image number 10 
checking image number 11 
checking image number 12 
checking image number 13 
checking image number 14 
checking image number 15 
checking image number 16 
checking image number 17 
checking image number 18 
checking image number 19 
checking image number 20 
checking image number 21 
checking image number 22 
checking image number 23 
checking image number 24 
checking image number 25 
checking image number 26 
checking image number 27 
checking image number 28 
checking image number 29 
checking image number 30 
checking image number 31 
checking image number 32 
checking image number 33 
checking image number 34 
checking image number 35 
checking image number 36 
checking image number 37 
checking image number 

Data Redcution: reduce the number of AI-generated images and human-genereated image to a balance state. due to space contrain on github, we gonna reduce the number of image to 50 using random sampling.

In [29]:
from data_preprocessing.data_reduction import Data_reduction
# if output "The number of images is less than the sampling size" that mean the dataset is already being reduced
Data_reduction = Data_reduction(50)
Data_reduction.random_sampling('ai_generated_images/')
Data_reduction.random_sampling('human_generated_images/')

50
The number of images is less than the sampling size
50
The number of images is less than the sampling size


Data labeling: lable the dataset for our supervised model

In [30]:
from data_preprocessing.data_labeling import Data_labeling
ai_data_labeling = Data_labeling()
#label ai-generated images as 0
ai_images, ai_label = ai_data_labeling.labeling('ai_generated_images/', 0)
human_data_labeling = Data_labeling()
#label human-generated images as 1
human_images, human_label = human_data_labeling.labeling('human_generated_images/', 1)
# For testing purpose we not going to save the data, using the exsiting npy file for train the model.
# np.save('ai_images.npy', ai_images)
# np.save('ai_label.npy', ai_label)
# np.save('human_images.npy', human_images)
# np.save('human_label.npy', human_label)
print('ai-generated image array shape: ',ai_images.shape) #(2999, 256, 256, 3)
print('ai-generated label array shape: ',ai_label.shape) #(2999,)
print('human-generated image array shape: ',human_images.shape)#(2999, 256, 256, 3)
print('human-generated label array shape: ',human_label.shape)#(2999,)

[[[ 55  90  94]
  [ 52  87  91]
  [ 50  85  89]
  ...
  [ 80  97  81]
  [ 86 103  85]
  [ 90 107  89]]

 [[ 53  88  92]
  [ 55  90  94]
  [ 58  93  97]
  ...
  [ 75  92  76]
  [ 79  96  80]
  [ 82  99  81]]

 [[ 50  85  89]
  [ 56  91  95]
  [ 62  97 101]
  ...
  [ 69  85  72]
  [ 70  87  71]
  [ 72  89  73]]

 ...

 [[ 83  84  70]
  [ 92  93  77]
  [110 109  91]
  ...
  [ 71  71  47]
  [ 75  75  51]
  [ 80  80  56]]

 [[ 67  65  50]
  [ 73  72  54]
  [ 73  70  51]
  ...
  [ 71  74  47]
  [ 75  78  51]
  [ 79  82  55]]

 [[ 94  93  75]
  [119 118  98]
  [134 131 112]
  ...
  [ 83  87  60]
  [ 87  90  63]
  [ 90  93  66]]]
[[[ 74  75  79]
  [ 73  74  78]
  [ 73  74  78]
  ...
  [ 63  65  86]
  [ 55  60  80]
  [ 47  52  72]]

 [[ 73  74  78]
  [ 73  74  78]
  [ 72  73  77]
  ...
  [ 51  53  74]
  [ 55  57  78]
  [ 53  58  78]]

 [[ 72  73  77]
  [ 71  72  76]
  [ 71  72  76]
  ...
  [ 42  45  64]
  [ 52  55  74]
  [ 58  61  80]]

 ...

 [[  9  10  12]
  [  9  10  12]
  [  9  10  12]
  ..